## Demo of 1D regression with an Attentive Neural Process (ANP) model

This notebook will provide a simple and straightforward demonstration on how to utilize an Attentive Neural Process (ANP) to regress context and target points to a sine curve.

First, we need to import all necessary packages and modules for our task:

In [ ]:
import os
import sys
import torch
from matplotlib import pyplot as plt

# Provide access to modules in repo.
sys.path.insert(0, os.path.abspath('neural_process_models'))
sys.path.insert(0, os.path.abspath('misc'))

from neural_process_models.attentive_neural_process import NeuralProcessModel
from misc.test_sin_regression.Sin_Wave_Data import sin_wave_data, plot_functions

The `sin_wave_data` class, defined in `misc/test_sin_regression/Sin_Data_Wave.py`, represents the curve that we will try to regress to. From instances of this class, we are able to sample context and target points from the curve to serve as inputs for our neural process.

The default parameters of this class will produce a "ground truth" curve defined as the sum of the following:
1. A sine curve with amplitude 1, frequency 1, and phase 1.
2. A sine curve with amplitude 2, frequency 2, and phase 1.
3. A measured amount of noise (0.1).

Let us create an instance of this class:

In [ ]:
data = sin_wave_data()

Next, we need to instantiate our model. The ANP model is implemented under the `NeuralProcessModel` class under the file `neural_process_models/attentive_neural_process.py`.

We will use the following parameters for our example model:
* 1 for x-dimension and y-dimension (since this is 1D regression)
* 4 hidden layers of dimension 256 for encoders and decoder
* 256 as the latent dimension for encoders and decoder
* We will utilize a self-attention process.
* We will utilize a deterministic path for the encoder.

Let us create an instance of this class, as well as set some hyperparameters for our training:

In [ ]:
np_model = NeuralProcessModel(x_dim=1,
                              y_dim=1,
                              mlp_hidden_size_list=[256, 256, 256, 256],
                              latent_dim=256,
                              use_rnn=False,
                              use_self_attention=True,
                              use_deter_path=True)

optim = torch.optim.Adam(np_model.parameters(), lr=1e-4)
num_epochs = 1000
batch_size = 16

Now, let us train our model. For each epoch, we will print the loss at that epoch. Here is a GIF displaying the regression results over the duration of training:

![SegmentLocal](misc/images/anp_-3_3.gif "results")

Additionally, every 100 epochs, an image will be generated and displayed, using `pyplot`. This will give you an opportunity to more closely analyze and/or save the images, if you would like.

In [ ]:
print("Training...")

for epoch in range(1, num_epochs + 1):
    print("step = " + str(epoch))

    np_model.train()

    plt.clf()
    optim.zero_grad()

    ctt_x, ctt_y, tgt_x, tgt_y = data.query(batch_size=batch_size,
                                            context_x_start=-6,
                                            context_x_end=6,
                                            context_x_num=200,
                                            target_x_start=-6,
                                            target_x_end=6,
                                            target_x_num=200)

    mu, sigma, log_p, kl, loss = np_model(ctt_x, ctt_y, tgt_x, tgt_y)

    print('loss = ', loss)
    loss.backward()
    optim.step()
    np_model.eval()
    
    if epoch % 50 == 0:
        plt.ion()
        plot_functions(tgt_x.numpy(),
                       tgt_y.numpy(),
                       ctt_x.numpy(),
                       ctt_y.numpy(),
                       mu.detach().numpy(),
                       sigma.detach().numpy())
        title_str = 'Training at epoch ' + str(epoch)
        plt.title(title_str)
        plt.pause(0.1)

plt.ioff()
plt.show()